In [ ]:
!jupyter kernelspec list

In [ ]:
!pip list

In [ ]:
!where python

In [10]:
pip show openai

Name: openai
Version: 1.51.2
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: C:\Users\Sai Krishna\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install openai

## Basic imports 

In [33]:
import os

In [34]:
os.environ["PINECONE_API_KEY"] = <<PINECONE_API_KEY>>
os.environ["COHERE_API_KEY"] = <<COHERE_API_KEY>>
os.environ["OPENAI_API_KEY"] = <<OPENAI_API_KEY>>

In [35]:
os.getenv('COHERE_API_KEY')

'mW5tqCRrbHHimfFZh3F95zjgX0jB3wGCa0pVjjfT'

In [36]:
from transformers import AutoTokenizer, AutoModel

In [37]:
def get_embeddings(text, model_name="bert-base-uncased"):
        #Load the pre-trained model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        print("Model and Tokenizer Loaded")
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

## Using bert-base-uncased

In [38]:
text = "today is saterday and it is a good day"
model_name="bert-base-uncased"
embeddings= get_embeddings(text, model_name)

Model and Tokenizer Loaded


## Document Loading using Langchain PyPDFLoader

In [39]:
from langchain.document_loaders import PyPDFLoader

In [40]:
def document_loader(filename):
        """
        Load and split the text from the file into manageable chunks.
        """
        try:
            loader = PyPDFLoader(filename)
            text = loader.load()
            return text
        except FileNotFoundError:
            print(f"File not found: {filename}")
            return []
        except Exception as e:
            print(f"other exception {str(e)}")
            return []

In [41]:
path = 'P:/GEN-AI/vector-db-pinecone/pinecone/Attachment_t2522042_BlueMoonRenewLease_Nov062023044105.pdf'
text = document_loader(path)

In [42]:
type(text)

list

##  splitting using langchain RecursiveCharacterTextSplitter  - List [TEXT]

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [44]:
def split_text(text):
        """
        Load and split the text from the file into manageable chunks.
        """
        try:
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=10000,
                chunk_overlap=100,
                is_separator_regex=False,
            )
            return splitter.split_text(str(text))
        except FileNotFoundError:
            print(f"File not found: {filename}")
            return []
        except Exception as e:
            print(f"other error : {str(e)}")
            return []

## Get splitting text here and store it in vector DB 

In [45]:
lang_chain_split_text = split_text (text)

In [46]:
type(lang_chain_split_text)

list

## Initialize pine cone 

In [47]:
from pinecone import Pinecone

In [48]:
def init_pinecone():
    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )
    return pc

## Connect to Pine cone index 

In [49]:
def connect_to_index(pinecone, index_name):
    index = pinecone.Index(index_name)
    return index

In [50]:
pinecone = init_pinecone()

## List of indexes in pinecone

In [51]:
pinecone.list_indexes().names()

['my-test-index',
 'sample',
 'name-of-your-index',
 'cohere-example',
 'fruit-animal-index',
 'documents']

## Know more about index 

In [52]:
#print(f"Index List :::: {pinecone.list_indexes()}")

In [53]:
index_name = "cohere-example"

## Get the index object

In [54]:
index = connect_to_index(pinecone,index_name)

In [55]:
index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1},
                'my-example-namespace': {'vector_count': 1},
                'new-example-namespace': {'vector_count': 46}},
 'total_vector_count': 48}

## Prepare data for insertion

In [56]:
import cohere

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Sai Krishna\AppData\Local\sagemaker\sagemaker\config.yaml


In [57]:
    #texts=["hello", "goodbye"], model="embed-english-v3.0", input_type="classification", embedding_types=["float"]


In [58]:
def get_embeddings(texts, model_name):
      
        # Initialize Cohere client with API key
        cohere_client = cohere.Client(os.getenv("COHERE_API_KEY"))
    
        # Convert text to embeddings using the specified model
        response = cohere_client.embed(texts=texts, model=model_name)

        # Extract embeddings from the response
        embeddings = response.embeddings
        return embeddings

## Models and dimentions
#####  embed-english-v3.0 <font color = green > 1024 </font>

#####  embed-multilingual-v3.0 <font color = green > 1024 </font>

##### embed-english-light-v3.0 <font color = green > 384 </font>

##### embed-multilingual-light-v3.0 <font color = green > 384 </font>

##### embed-english-v2.0 <font color = green > 4096 </font>

##### embed-english-light-v2.0 <font color = green > 1024 </font> 

##### embed-multilingual-v2.0 <font color = green > 768 </font>

In [59]:
model_name="embed-english-v2.0"

In [60]:
embedded_data =  get_embeddings(lang_chain_split_text, model_name= model_name)

In [61]:
len(embedded_data)

46

In [62]:
import hashlib

In [63]:
def create_unique_id(filename):
    """
    Create a unique identifier for the file using the filename.
    """
    try:
        base_filename = os.path.basename(filename)
        filename_hash = hashlib.md5(base_filename.encode()).hexdigest()
        return f"{base_filename}_{filename_hash[:8]}"
    except Exception as e:
        logging.error(f"Failed to create a unique ID for the file: {str(e)}")
        return None

In [64]:
file_id = create_unique_id(path)
file_name = 'Attachment_t2522042_BlueMoonRenewLease_Nov062023044105.pdf'

###  upsert/ insert vectors into pinecone
#####      1. Iterating embedding data which is in th form of LIST 
#####      2. form the Unique_id ( file_id+'chunk'_chunk_index) 
#####      3. inserted in namespace with metadata 
       

In [65]:
namespace= "new-example-namespace"
metadata = {"source": file_name, "file_id": file_id}

In [66]:
def is_existing(unique_id, namespace):
    existing_vector = index.fetch(ids=[unique_id], namespace=namespace)
    if existing_vector.get('vectors'):
        return True
    else:
        return False       

In [67]:
for chunk_index , chunk in enumerate(embedded_data):
    unique_id = f"{file_id}_chunk_{chunk_index}"
    if not is_existing(unique_id,namespace):
        vectors = [(unique_id, chunk , metadata)]
        #index.upsert(vectors=vectors,  namespace=namespace)
        print(f'Successfully inserted {file_name} with {len(embedded_data)} chunks ' )
    else:
        pass
       #print('Already exist in vector DB ')

### <font color = blue >upsert and Similarity seach fruit and animal </font>

In [68]:
import openai

In [69]:
fruit_index = 'fruit-animal-index'
fruit_index = connect_to_index(pinecone,fruit_index)

In [70]:
# Set up your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")  # You can set the key directly if not using environment variables

In [71]:
# Sample text to embed
texts = ['apple', 'orange', 'grape','cat','dog','cow','Elephant','rat','Banana']

In [93]:
def get_embeddings(text):
     response = openai.embeddings.create( model="text-embedding-ada-002", input=text)
     embedding = response.data[0].embedding
     return embedding
    

In [94]:
for id,text in enumerate(texts):
    # Get the embeddings using the updated API
    embedding = get_embeddings(text)
    vectors = [(str(id), embedding )]
    fruit_index.upsert(vectors=vectors,  namespace='fruits-animals')
    print('upsert is done ')


    


upsert is done 
upsert is done 
upsert is done 
upsert is done 
upsert is done 
upsert is done 
upsert is done 
upsert is done 
upsert is done 


In [106]:
# Perform a similarity search
query = "rat"
query_embedding = get_embeddings(query)
query_embedding

[-0.0017083240672945976,
 -0.012699521146714687,
 0.0017343476647511125,
 -0.037984397262334824,
 -0.007232884410768747,
 -0.0029096074867993593,
 -0.017125213518738747,
 -0.026486996561288834,
 -0.027077985927462578,
 -0.011222051456570625,
 0.02183968387544155,
 0.005933383014053106,
 0.00021396024385467172,
 -0.013673308305442333,
 -0.005120774731040001,
 0.017917674034833908,
 0.03739340975880623,
 0.011228767223656178,
 0.026164639741182327,
 0.002573818899691105,
 -0.011383230797946453,
 -0.006403486710041761,
 -0.0074880835600197315,
 -0.014022528193891048,
 -0.01752815954387188,
 -0.01719237118959427,
 0.01904592476785183,
 -0.010483317077159882,
 0.008717069402337074,
 -0.008757364004850388,
 0.014049391262233257,
 -0.017098350450396538,
 -0.03782321885228157,
 -0.035136908292770386,
 -0.025157274678349495,
 -0.002038236241787672,
 0.0008780869538895786,
 0.004721186589449644,
 0.011940639466047287,
 0.0020264836493879557,
 0.021517327055335045,
 0.004418976604938507,
 0.01286

In [107]:
response = fruit_index.query(
    vector=query_embedding,  # Use 'vector' keyword for the embedding
    top_k=10,                 # Specify the number of top matches
    include_metadata=True     # Include metadata in the response
)

In [108]:
response

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

In [109]:
for match in response['matches']:
    score = match['score']
    tokens = 10# match['metadata'].get('tokens', 'No tokens available')  # Safely handle missing tokens
    print(f"Score: {score:.2f}, Tokens: {tokens}")

In [ ]:
# Initialize Cohere Embeddings
from cohere import Client as CohereClient

In [ ]:
# Load the API key and the model name
cohere_api_key = os.getenv("COHERE_API_KEY")
model_name = "embed-english-v2.0"  # Example model name, replace with your desired model.

# Initialize the CohereEmbeddings object
cohere_embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, model=model_name)
    

In [ ]:
# Perform a similarity search
query = "Move-Out Procedures"
query_embedding = cohere_embeddings.embed_query(query)

In [ ]:
response = index.query(
    vector=query_embedding,  # Use 'vector' keyword for the embedding
    top_k=2,                 # Specify the number of top matches
    include_metadata=True     # Include metadata in the response
)

In [ ]:
response['matches']

In [ ]:
for match in response['matches']:
    score = match['score']
    tokens = 10# match['metadata'].get('tokens', 'No tokens available')  # Safely handle missing tokens
    print(f"Score: {score:.2f}, Tokens: {tokens}")

## CHAT MODEL EXAMPLE 

In [ ]:
import json
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_cohere import ChatCohere, CohereEmbeddings, CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
import cohere
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os
import time


In [ ]:
# Initialize Cohere embeddings
cohere_embeddings = CohereEmbeddings(model=model_name, cohere_api_key=os.getenv('COHERE_API_KEY'))

In [ ]:
# Initialize the language model
model_id = "command-r-plus-08-2024"
llm = ChatCohere(model=model_id, cohere_api_key=os.getenv('COHERE_API_KEY'), temperature=0.1)

In [ ]:
# Set up the document search with Pinecone vector store
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=cohere_embeddings, text_key="text")


In [ ]:
def create_prompt(question):
    prompt_template = """
                Please provide a short and direct answer from the context below, repeating the exact phrasing of the question in your response.
                For example, if the question is 'What is the borrower name?', respond with 'Borrower name: [exact answer]'.
                If the answer cannot be found, respond with 'Apologies, but I couldn't find the information you requested in the document.'.
                Do not make up any answer.
                Context:
                {context}
                """
 
    messages = [
        SystemMessagePromptTemplate.from_template(prompt_template),
        HumanMessagePromptTemplate.from_template(question)
    ]
    prompt = ChatPromptTemplate.from_messages(messages)
    print(prompt)
    return prompt
     
   

In [ ]:
while True:
    user_question = input(
        "Please enter your question (or type 'exit' to quit): "
    ).strip()
 
    if user_question.lower() == "exit":  # Exit condition
        break

    # Generate and print the response
    prompt = create_prompt(user_question)
    chain_type_kwargs = {"prompt": prompt}
    t1=time.time()
    top_k=30
    retriever = docsearch.as_retriever(
            include_metadata=True,
            metadata_key='source',
            top_k=30,
            search_type="similarity",
            search_kwargs={"filter": {"file_id": file_id}}
        )
    compressor = CohereRerank(model=model_name,top_n=8)
    compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
    )
    # compressed_docs = compression_retriever.get_relevant_documents(user_question)
    # Print the relevant documents from using the embeddings and reranker
    # print(compressed_docs)
    t2=time.time()
    print(t2-t1,"retriever time****")
 
    chain = RetrievalQA.from_chain_type(
    llm=llm,    chain_type="stuff", retriever=compression_retriever,return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
    )
    t3=time.time()
    answer=chain.invoke({"query": user_question})
    # print("response",answer)
    t4=time.time()
    print(t4-t3,"time at chain.invoke")
    # print(answer)
    print("Result",answer["result"])
 